In [6]:
%load_ext autoreload
%autoreload 2
from histopolalign.VerificationAlignment import verify_alignment_procedure
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# dice_mean, dice_std, imgs = verify_alignment_procedure.verify_alignment_main()
dice_scores, assd_scores, hausdorff_distances = verify_alignment_procedure.verify_alignment_main()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:45<00:00, 40.55s/it]


In [8]:
dice_scores

[{'raw': 0.313637226906058,
  'pre-align': 0.5985928389810603,
  'contour': 0.6214834154086175,
  'MLS': 0.7044732323552081},
 {'raw': 0.09420914453899304,
  'pre-align': 0.1150255157985273,
  'contour': 0.10878165723163304,
  'MLS': 0.09022327312188981}]

In [9]:
assd_scores

[{'raw': 41.2083302676335,
  'pre-align': 13.368728555258997,
  'contour': 10.164626309150075,
  'MLS': 9.738845458282109},
 {'raw': 23.277291870623326,
  'pre-align': 3.0614977680874746,
  'contour': 2.5102141495284016,
  'MLS': 2.770482662355907}]

In [10]:
hausdorff_distances

[{'raw': 108.94847317996712,
  'pre-align': 60.487755692728854,
  'contour': 57.19331672737279,
  'MLS': 45.0507411813452},
 {'raw': 34.12081757159383,
  'pre-align': 37.03689257020285,
  'contour': 38.20765004187539,
  'MLS': 41.39239929034614}]